## So you want to stream CryptoCurrency data?
Most exchanges provide a public websocket service that sends real-time market data to you. You can use this free service to automate trading or perform quick calculations that are not easy to perform on the exchange's charts. This data arrives in massive amounts, sometimes making real-time processing of the data difficult.

In this tutorial, I will show you how to subscribe to a websocket on Binance, and then we will do some cool things with the stream. 

**What we will accomplish:**
1. Sign up for Binance and get your API key
2. Set up a websocket connection to Binance
3. Process the data into a useful form
4. Perform some calculations on realtime data
5. Look at a better way of accomplishing the same thing

### Sign up for Binance & Get API Keys

The first step is to get yourself [signed up on binance](https://www.binance.com/register.html?ref=16036057).

<img src="signup-binance.png" width="500">

Once you have signed up, visit your account and click **API Setting**

<img src="api_step1.png">

Then, go ahead and name your API key. We'll call it binance-websocket-tutorial for now. Because that is what this is, afterall. 

<img src="api_step2.png">

After you verify, you will see a screen that has your API key and API Secret. You wont be able to see your secret key again, so make note of it. 

<img src="store_api_key.png">

### Set up a websocket connection to Binance

Sammchardy on github has provided an amazing library to interface with Binance's api. First you're going to have to install the [python-binance wrapper](https://github.com/sammchardy/python-binance) from github like so:

`pip3 install python-binance`

Sockets are handled through the BinanceSocketManager. It can handle multiple socket connections. When creating a socket connection, a _callback_ function is passed which receives the messages. Lets jump right into an example. We are going to listen to the websocket for the symbol `ETHBTC`. We want to see every trades as it occurs, and we can do so using Binance's trade socket.

#### Use the following code sample

In [1]:
import time
from binance.client import Client # Import the Binance Client
from binance.websockets import BinanceSocketManager # Import the Binance Socket Manager

# Although fine for tutorial purposes, your API Keys should never be placed directly in the script like below. 
# You should use a config file (cfg or yaml) to store them and reference when needed.
PUBLIC = '<YOUR-PUBLIC-KEY>'
SECRET = '<YOUR-SECRET-KEY>'

# Instantiate a Client 
client = Client(api_key=PUBLIC, api_secret=SECRET)

# Instantiate a BinanceSocketManager, passing in the client that you instantiated
bm = BinanceSocketManager(client)

# This is our callback function. For now, it just prints messages as they come.
def handle_message(msg):
    print(msg)

# Start trade socket with 'ETHBTC' and use handle_message to.. handle the message.
conn_key = bm.start_trade_socket('ETHBTC', handle_message)
# then start the socket manager
bm.start()

# let some data flow..
time.sleep(10)

# stop the socket manager
bm.stop_socket(conn_key)

{'e': 'trade', 'E': 1529623973721, 's': 'ETHBTC', 't': 69713886, 'p': '0.07836000', 'q': '0.03000000', 'b': 169884767, 'a': 169884759, 'T': 1529623973718, 'm': False, 'M': True}
{'e': 'trade', 'E': 1529623973721, 's': 'ETHBTC', 't': 69713887, 'p': '0.07836100', 'q': '0.03600000', 'b': 169884767, 'a': 169884580, 'T': 1529623973718, 'm': False, 'M': True}
{'e': 'trade', 'E': 1529623974326, 's': 'ETHBTC', 't': 69713888, 'p': '0.07830500', 'q': '0.15600000', 'b': 169884761, 'a': 169884769, 'T': 1529623974324, 'm': True, 'M': True}


**Congratulations - You're now streaming cryptocurrency data!**

### Process the data into a useful form

As you saw earlier, we were streaming real trades from Binance. But, we didn't get the most useful data printing out. That is because our handle_message function was fairly simple. Let's work on it a bit to create something more useful. 

The trade socket returns a dictionary as specified in [Binance's official api documentation](https://github.com/binance-exchange/binance-official-api-docs/blob/master/web-socket-streams.md) for every trade that occurs. We expect the payload to look like this: 

`{
  "e": "trade",     // Event type
  "E": 123456789,   // Event time
  "s": "BNBBTC",    // Symbol
  "t": 12345,       // Trade ID
  "p": "0.001",     // Price
  "q": "100",       // Quantity
  "b": 88,          // Buyer order Id
  "a": 50,          // Seller order Id
  "T": 123456785,   // Trade time
  "m": true,        // Is the buyer the market maker?
  "M": true         // Ignore.
}`

In the event of an error, the BinanceSocketManager tries reconnecting a maximum of 5 times. If it fails to do so, we should expect a message like this:

`{
    'e': 'error',
    'm': 'Max reconnect retries reached'
}`

With all of that in mind, let's create a smarter _callback function_ that will define what to do when we get a trade message. We will call it **handle_message**

In [2]:
def handle_message(msg):
    
    # If the message is an error, print the error
    if msg['e'] == 'error':    
        print(msg['m'])
    
    # If the message is a trade: print time, symbol, price, and quantity
    else:
        print("Time: {} Symbol: {} Price: {} Quantity: {} ".format(msg['T'], msg['s'], msg['p'], msg['q']))

We will again specify that we want to listen to `ETHBTC`, and we want **handle_message** to deal with the trades as they come. Lets try this again.

In [4]:
import time
from binance.client import Client 
from binance.websockets import BinanceSocketManager

PUBLIC = '<YOUR-PUBLIC-KEY>'
SECRET = '<YOUR-SECRET-KEY>'

client = Client(api_key=PUBLIC, api_secret=SECRET)
bm = BinanceSocketManager(client)

conn_key = bm.start_trade_socket('ETHBTC', handle_message)

bm.start()

time.sleep(10) # let some data flow..

bm.stop_socket(conn_key)

Time: 1529624001948 Symbol: ETHBTC Price: 0.07829500 Quantity: 0.00100000 
Time: 1529624001948 Symbol: ETHBTC Price: 0.07829200 Quantity: 0.03800000 
Time: 1529624001948 Symbol: ETHBTC Price: 0.07829100 Quantity: 0.01600000 
Time: 1529624002093 Symbol: ETHBTC Price: 0.07829100 Quantity: 0.04600000 
Time: 1529624002363 Symbol: ETHBTC Price: 0.07834700 Quantity: 0.16700000 
Time: 1529624004165 Symbol: ETHBTC Price: 0.07829500 Quantity: 0.10100000 
Time: 1529624004184 Symbol: ETHBTC Price: 0.07829200 Quantity: 0.07600000 
Time: 1529624004217 Symbol: ETHBTC Price: 0.07829200 Quantity: 1.28800000 
Time: 1529624004292 Symbol: ETHBTC Price: 0.07829200 Quantity: 0.56800000 


### Perform Some Calculations on Realtime Data

We can make our handle_message even smarter. Let's say we were interested in how many bitcoins were exchanging hands per ETHBTC trade. We need to do a few things:

- Each trade has a price and a quantity. We can multiply that to get the amount of bitcoin traded.
- Since humans don't respond with "1529616742353 milliseconds since Thursday, 1 January 1970" when asked for the time, we'll make the time format human readable. 
- It would also be usefull to know if the trade was a buy or a sell.

#### First lets determine how many bitcoins were exchanged per ETHBTC trade. 

In [1]:
def handle_message(msg):
    
    if msg['e'] == 'error':    
        print(msg['m'])
        
    else:
        # Bitcoins exchanged, is equal to price times quantity
        bitcoins_exchanged = msg['p'] * msg['q']
        
        # Print this amount
        print("Time: {} Symbol: {} Price: {} Quantity: {} BTC Exchanged: {}".format(msg['T'], 
                                                                                    msg['s'], 
                                                                                    msg['p'], 
                                                                                    msg['q'],
                                                                                   bitcoins_exchanged))

If we run this again...

In [ ]:
import time
from binance.client import Client 
from binance.websockets import BinanceSocketManager

PUBLIC = '<YOUR-PUBLIC-KEY>'
SECRET = '<YOUR-SECRET-KEY>'

client = Client(api_key=PUBLIC, api_secret=SECRET)
bm = BinanceSocketManager(client)

conn_key = bm.start_trade_socket('ETHBTC', handle_message)

bm.start()

time.sleep(10) # let some data flow..

bm.stop_socket(conn_key)

We run into the following error: `builtins.TypeError: can't multiply sequence by non-int of type 'str'`

This is a funny error. We would expect price and quantity to come back as numbers, but for some reason they are actually coming to us from BinanceSocketManager as strings. If you look at schema of the dictionary that is returned, you can see for yourself:

`"p": "0.001",     // Price`

`"q": "100",       // Quantity`

I don't know whose decision that was... but, it's a good thing we caught that. Let's try again by turning price and quantity into floats.

In [3]:
def handle_message(msg):
    
    if msg['e'] == 'error':    
        print(msg['m'])
        
    else:
        # Bitcoins exchanged - This time converting the strings to floats.
        bitcoins_exchanged = float(msg['p']) * float(msg['q'])
        
        # Print this amount
        print("Time: {} - Symbol: {} - Price: {} - Quantity: {} BTC Quantity: {}".format(msg['T'], 
                                                                                    msg['s'], 
                                                                                    msg['p'], 
                                                                                    msg['q'],
                                                                                   bitcoins_exchanged))

In [4]:
import time
from binance.client import Client 
from binance.websockets import BinanceSocketManager

PUBLIC = '<YOUR-PUBLIC-KEY>'
SECRET = '<YOUR-SECRET-KEY>'

client = Client(api_key=PUBLIC, api_secret=SECRET)
bm = BinanceSocketManager(client)

conn_key = bm.start_trade_socket('ETHBTC', handle_message)

bm.start()

time.sleep(10) # let some data flow..

bm.stop_socket(conn_key)

Time: 1529624811880 - Symbol: ETHBTC - Price: 0.07837300 - Quantity: 4.65200000 BTC Quantity: 0.364591196
Time: 1529624812169 - Symbol: ETHBTC - Price: 0.07833700 - Quantity: 0.10000000 BTC Quantity: 0.0078337
Time: 1529624814290 - Symbol: ETHBTC - Price: 0.07833700 - Quantity: 0.90100000 BTC Quantity: 0.070581637
Time: 1529624814386 - Symbol: ETHBTC - Price: 0.07837400 - Quantity: 0.12700000 BTC Quantity: 0.009953498
Time: 1529624814451 - Symbol: ETHBTC - Price: 0.07833700 - Quantity: 0.05000000 BTC Quantity: 0.00391685
Time: 1529624815140 - Symbol: ETHBTC - Price: 0.07833700 - Quantity: 0.00300000 BTC Quantity: 0.000235011
Time: 1529624815481 - Symbol: ETHBTC - Price: 0.07833700 - Quantity: 0.03000000 BTC Quantity: 0.00235011
Time: 1529624815481 - Symbol: ETHBTC - Price: 0.07829900 - Quantity: 0.02500000 BTC Quantity: 0.0019574749999999998
Time: 1529624818106 - Symbol: ETHBTC - Price: 0.07830400 - Quantity: 0.06300000 BTC Quantity: 0.004933152
Time: 1529624821257 - Symbol: ETHBTC - P

#### Now, lets make that time a little bit prettier

In [5]:
# Binance returns a UTC timestamp in milliseconds. 
timestamp = 1529618149064
# Convert this timestamp into seconds by dividing by 1000
timestamp = timestamp / 1000
# Use the datetime library to convert this into a datetime
from datetime import datetime
new_time = datetime.fromtimestamp(timestamp)
# Use strftime to make it readable
new_time.strftime('%Y-%m-%d %H:%M:%S')

'2018-06-21 14:55:49'

#### Now, lets find out if this was a buy or sell event

In our dictionary, we are given a boolean if the buyer is the market maker.

`"m": true,        // Is the buyer the market maker?`

As a refresher, the market maker adds liquidity to the market. If the buyer is a market maker, it means that someone has bought the asset from at his or her price. Therefore, we can classify this as a 'sell' event. 

#### All together

In [1]:
from binance.client import Client
from binance.websockets import BinanceSocketManager 
from datetime import datetime
import time
# Although fine for tutorial purposes, your API Keys should never be placed directly in the script like below. 
# You should use a config file (cfg or yaml) to store them and reference when needed.
PUBLIC = '<YOUR-PUBLIC-KEY>'
SECRET = '<YOUR-SECRET-KEY>'

client = Client(api_key=PUBLIC, api_secret=SECRET)
bm = BinanceSocketManager(client)

def handle_message(msg):
    
    if msg['e'] == 'error':    
        print(msg['m'])
        
    else:
        # Bitcoins exchanged - This time converting the strings to floats.
        bitcoins_exchanged = float(msg['p']) * float(msg['q'])
        
        # Make time pretty
        timestamp = msg['T'] / 1000
        timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
        
        # Buy or sell?
        if msg['m'] == True:
            event_side = 'SELL'
        else:
            event_side = 'BUY '
        
        # Print this amount
        print("{} - {} - {} - Price: {} - Qty: {} BTC Qty: {}".format(timestamp,
                                                                       event_side,
                                                                       msg['s'],
                                                                       msg['p'],
                                                                       msg['q'],
                                                                       bitcoins_exchanged))

conn_key = bm.start_trade_socket('ETHBTC', handle_message)

bm.start()

time.sleep(5) # let some data flow..

bm.stop_socket(conn_key)

2018-06-21 16:48:20 - SELL - ETHBTC - Price: 0.07833300 - Qty: 0.01900000 BTC Qty: 0.001488327
2018-06-21 16:48:20 - SELL - ETHBTC - Price: 0.07832600 - Qty: 0.04200000 BTC Qty: 0.0032896920000000003
2018-06-21 16:48:23 - SELL - ETHBTC - Price: 0.07832700 - Qty: 0.06200000 BTC Qty: 0.004856274


### A Better Way
What we did above was pretty cool. We subscribed to a stream of realtime trade data from binance, made it easy to read, and performed a simple calculation on it. But there was a lot of work that we had to do to get there. What if we had to do this for every exchange? A lot of time evaluating the market would be spent getting stuck on issues like dealing with numbers as strings, or cryptic 'is buyer market maker' type fields. Every exchange is different too. 

We also need to keep in mind that we only start getting new trades. We don't even have access to historical data yet for binance.

### LiveDataFrame
The good news is, LiveDataFrame has taken care of that for you. It provides access to every coin on Binance (and other exchanges too!), with realtime updates of OHLC data with 5 second granularity. It also provides up to 4 hours of historical data for every coin!

Lets see how easy it is to start with LiveDataFrame. First, [sign up](https://app.livedataframe.com/users/new) and get your api keys.

Then `pip install livedataframe` to install the client and use the code below:

In [2]:
from livedataframe import LiveExchange, ExchangeInfo

# Lets get a list of all symbols trading on binance
all_symbols_binance = ExchangeInfo.list_symbols_for('binance')

livedf = LiveExchange(
    public_key='<YOUR-PUBLIC-KEY>', # Enter your public key that was emailed
    secret_key='<YOUR-PRIVATE-KEY>', # Enter your secret key that was emailed
    exchange = 'binance', # Enter your exchange
    symbols = all_symbols_binance, # Enter a dict of symbols. How about lets get them all?
    lookback_period = '1H')

livedf.start()

Seeding With Data... This Could Take a Few Minutes.
TOTAL TIME TO LOAD: 0:01:02.963685


You can access a symbol that is trading on binance using the symbols dictionary. What you get is a [DataFrame](https://www.datacamp.com/community/blog/python-pandas-cheat-sheet) that is always up to date with the current OHLC data on binance. Not bad for 15 lines of code.

In [3]:
livedf.symbols['ETHBTC'].tail()

,symbol,price_change,price_change_pct,weighted_avg_price,last_price,close,open,high,low,prev_day_close_price,last_trade_qty,bid_price,bid_qty,ask_price,ask_qty,base_volume,volume
2018-06-22 05:50:18+00:00,ETHBTC,-0.001144,-1.442,0.079143,0.078197,0.078197,0.079341,0.080394,0.077853,0.079341,0.070,0.078198,0.951,0.078258,0.063,102228.606,8090.698545
2018-06-22 05:50:23+00:00,ETHBTC,-0.001077,-1.358,0.079143,0.078258,0.078258,0.079335,0.080394,0.077853,0.079342,0.062,0.078202,0.036,0.078258,0.001,102224.116,8090.342250
2018-06-22 05:50:28+00:00,ETHBTC,-0.001146,-1.444,0.079143,0.078201,0.078201,0.079347,0.080394,0.077853,0.079285,0.038,0.078201,0.025,0.078256,0.539,102199.369,8088.378833
2018-06-22 05:50:33+00:00,ETHBTC,-0.001098,-1.384,0.079143,0.078250,0.078250,0.079348,0.080394,0.077853,0.079348,1.323,0.078203,0.292,0.078250,1.433,102199.567,8088.393092
2018-06-22 05:50:38+00:00,ETHBTC,-0.001148,-1.447,0.079143,0.078205,0.078205,0.079353,0.080394,0.077853,0.079350,0.347,0.078205,1.579,0.078261,0.036,102201.004,8088.502077
2018-06-22 05:50:43+00:00,ETHBTC,-0.001093,-1.377,0.079143,0.078262,0.078262,0.079355,0.080394,0.077853,0.079355,0.840,0.078211,2.010,0.078263,14.998,102196.658,8088.156211
2018-06-22 05:50:48+00:00,ETHBTC,-0.001096,-1.381,0.079143,0.078263,0.078263,0.079359,0.080394,0.077853,0.079354,2.602,0.078220,0.200,0.078263,12.396,102188.858,8087.532103
2018-06-22 05:50:53+00:00,ETHBTC,-0.001095,-1.380,0.079143,0.078263,0.078263,0.079358,0.080394,0.077853,0.079359,12.479,0.078220,0.200,0.078263,2.519,102187.587,8087.420441
2018-06-22 05:50:58+00:00,ETHBTC,-0.001095,-1.380,0.079143,0.078263,0.078263,0.079358,0.080394,0.077853,0.079358,14.260,0.078220,0.200,0.078263,0.738,102185.835,8087.279452
2018-06-22 05:51:03+00:00,ETHBTC,-0.001135,-1.430,0.079143,0.078224,0.078224,0.079359,0.080394,0.077853,0.079359,0.061,0.078220,0.200,0.078224,0.002,102176.704,8086.549900


Lets look at last price and last trade quantity:

In [4]:
#      access 'ETHBTC'    'last price'  'last trade qty'    '-1 for most recent value'
livedf.symbols['ETHBTC'][['last_price', 'last_trade_qty']].iloc[[-1]]

,last_price,last_trade_qty
2018-06-22 06:50:24+00:00,0.078322,0.055


Very easy to read isn't it? Lets do some magic and watch this in real time:

In [6]:
import time
from IPython.display import clear_output # To clear the output.
  
while True:
    try: 
        clear_output(wait=True)
        print(livedf.symbols['ETHBTC'][['last_price', 'last_trade_qty']].iloc[[-1]])
        time.sleep(5)


                           last_price  last_trade_qty
2018-06-22 06:52:19+00:00    0.078287           0.062


More...

In [7]:
import time
from IPython.display import clear_output # To clear the output.

while True:
    clear_output(wait=True)
    print(livedf.symbols['ETHBTC'][['symbol', 'last_price', 'last_trade_qty']].iloc[[-1]])
    print('\n')
    print(livedf.symbols['LTCBTC'][['symbol', 'last_price', 'last_trade_qty']].iloc[[-1]])
    time.sleep(5)

                           symbol  last_price  last_trade_qty
2018-06-22 06:52:49+00:00  ETHBTC    0.078365           0.544


                           symbol  last_price  last_trade_qty
2018-06-22 06:52:49+00:00  LTCBTC    0.014386             9.4


Check out our [documentation](https://docs.livedataframe.com) to see what else you can do with LiveDataFrame!

_DISCLAIMER: The above references an opinion and is for information purposes only. It is not intended to be investment advice. Seek a duly licensed professional for investment advice._